In [1]:
from dask.distributed import Client
import dask.dataframe as df

client = Client(processes=False, threads_per_worker=2,
                n_workers=3, memory_limit='4GB')
client

/home/kashif/anaconda3/lib/python3.7/site-packages/distributed/bokeh/core.py:57: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn('\n' + msg)


Client Scheduler: inproc://192.168.0.105/20013/1 Dashboard: http://localhost:38115/status,Cluster Workers: 3 Cores: 6 Memory: 12.00 GB


In [2]:
import time

def some_func(x):
    time.sleep(1)
    y = x + 1
    time.sleep(1)
    z = x * y
    time.sleep(1)
    return z

## `Client.submit` returns a Future Object

In [3]:
result_1 = client.submit(some_func, 5)  

In [5]:
result_1

<Future: status: finished, type: int, key: some_func-69b9ca2561c3e2c7a2601755ca473865>

## In this case the computation will be pending

In [6]:
def some_func_long(x):
    time.sleep(10)
    y = x + 1
    time.sleep(5)
    z = x * y
    time.sleep(2)
    return z

In [7]:
result_2 = client.submit(some_func_long, 5)  

In [9]:
result_2

<Future: status: pending, key: some_func_long-2e59f70fa9cadecab89bf7b4648344e6>

## `result()` function can be used to force the computation to complete

In [10]:
result_1.result()

30

In [14]:
result_3 = client.submit(some_func_long, 15)  

In [15]:
result_3.result()

240

## We can evaluate multiple Future objects in one go using `gather`

In [19]:
futures_list = [client.submit(some_func, x)
               for x in range(1, 8)]

In [20]:
all_res = client.gather(futures_list)

In [21]:
all_res

[2, 6, 12, 20, 30, 42, 56]

## We can also use `map` function to pass multiple future object

In [22]:
list_result = client.map(some_func_long, range(5))

In [23]:
client.gather(list_result)

[0, 2, 6, 12, 20]

### Reference : http://docs.dask.org/en/latest/futures.html